In [1]:
import numpy as np

In [2]:
with open("LR_Q1_Dataset.txt") as f:
    l=f.readlines()
    f.close()
exam_score_1 = []
exam_score_2 = []
y=[]
for i in l:
    k=i.split(',')
    exam_score_1.append(k[0])
    exam_score_2.append(k[1])
    y.append(k[2][0])

In [3]:
exam_score_1 = np.array(exam_score_1,dtype=float)
exam_score_2 = np.array(exam_score_2,dtype=float)
y = np.array(y,dtype=float)

In [4]:
#APPLYING MEAN NORMALIZATION AND FEATURE SCALING
mean_score1 = np.mean(exam_score_1)
min_score1 =np.min(exam_score_1) 
max_score1 =np.max(exam_score_1)

In [5]:
mean_score2 = np.mean(exam_score_2)
min_score2 =np.min(exam_score_2) 
max_score2 =np.max(exam_score_2)

In [6]:
mean_score1,mean_score2,min_score1,max_score1,min_score2,max_score2

(65.64427405732314,
 66.22199808811695,
 30.05882244669796,
 99.82785779692128,
 30.60326323428011,
 98.86943574220611)

# Dividing the dataset in training and testing and applying feature scaling

In [7]:
dataset_train=[]
y_train=[]
for i in range(70):
    dataset_train.append([1,(exam_score_1[i]-mean_score1)/(max_score1-min_score1),(exam_score_2[i]-mean_score2)/(max_score2-min_score2)])
    y_train.append(y[i])
dataset_train = np.array(dataset_train)
y_train=np.array(y_train)
#dataset_train,y_train

In [8]:
dataset_test=[]
y_test=[]
for i in range(70,100):
    dataset_test.append([1,(exam_score_1[i]-mean_score1)/(max_score1-min_score1),(exam_score_2[i]-mean_score2)/(max_score2-min_score2)])
    y_test.append(y[i])
datset_test = np.array(dataset_test)
y_test=np.array(y_test)
#dataset_test,y_test

## SIGMOID
$$ \ sigmoid(Z) = \frac{1}{(1+e^{-Z})} $$

## GRADIENT DESENT
$$ J(\theta) = -\frac{1}{m} \sum \limits _{i=1}^{m} y^i \log(h_ \theta(x^i))+(1-y^i)\log(1-h_\theta(x^i)) $$
$$  \theta(j) := \theta(j) - \sum \limits _{i=1}^{m}\frac{\partial J(\theta)}{\partial\theta} $$
$$  \frac{\partial J(\theta)}{\partial\theta} = (sigmoid(h_\theta(x^i))- y^i)x_i(j)$$

In [9]:
def sigmoid(z):
    return 1.0/(1+np.exp(-1*z))

In [10]:
def slope(Coeff, X_train, Y_train, ind):
    diff = 0
    for i in range(len(X_train)):
        h_theta = 0
        for j in range(len(Coeff)):
            h_theta = h_theta + Coeff[j] * X_train[i][j]
        diff += (sigmoid(h_theta) - Y_train[i]) * X_train[i][ind]
    return diff

In [11]:
def batchGradient(x_train,y_train,alpha = 0.000001,epoch=50000):
    coeff=[0,0,0]
    for i in range(epoch):
        tmp_coeff = coeff.copy()
        for j in range(len(coeff)):
            tmp_coeff[j] = tmp_coeff[j]-alpha*(slope(coeff,x_train,y_train,j))
        coeff = tmp_coeff.copy()
    return coeff

In [12]:
def printaccuracy(X_test, Y_test, Coeff):
    count = 0
    for i in range(len(X_test)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * X_test[i][j]
        predicted = sigmoid(predicted)
        if predicted > 0.5:
            if Y_test[i] == 1:
                count += 1
        else:
            if Y_test[i] == 0:
                count += 1
    print("Accuracy is : "+ str(count / len(Y_test) * 100))
    return count / len(Y_test) * 100

In [13]:
def stochaisticSlope(coeff,x_train,y_train,ind):
    diff=0
    for i in range(len(coeff)):
        diff = diff + coeff[i] * x_train[i]
    return (sigmoid(diff)-y_train)*x_train[ind]

In [14]:
def stochaisticGradient(X_train,Y_train,alpha=0.00001,epochs=5000):
    Coeff = [0, 0, 0]
    for iter in range(epochs):
        for i in range(len(Y_train)):
            TempCoeff = Coeff.copy()
            for j in range(len(TempCoeff)):
                TempCoeff[j] = TempCoeff[j] - (alpha * (stochaisticSlope(Coeff, X_train[i], Y_train[i], j)))
            Coeff = TempCoeff.copy()
    return Coeff

In [15]:
def minibtchgra(X_train, Y_train, alpha = 0.00001, epochs = 30, batchsize = 20):
    LearningRateScaling = alpha
    Coeff = [0, 0, 0]
    NoOfBatches = int(np.ceil(len(Y_train) / batchsize))
    equallyDiv = False
    if (len(Y_train) % batchsize == 0):
        equallyDiv = True;

    for epoch in range(epochs):
        for batch in range(NoOfBatches):
            Summation = [0, 0, 0]
            for j in range(len(Coeff)):
                for i in range(batchsize):
                    if (batch * batchsize + i == len(X_train)):
                        break
                    PredictedValue = 0.0
                    for wj in range(len(Coeff)):
                        PredictedValue += Coeff[wj] * X_train[batch * batchsize + i][wj]
                    PredictedValue = sigmoid(PredictedValue)
                    PredictedValue -= Y_train[batch * batchsize + i]
                    PredictedValue *= X_train[batch * batchsize + i][j]
                    Summation[j] += PredictedValue;

            if (not equallyDiv and batch == NoOfBatches - 1):
                for j in range(len(Summation)):
                    Coeff[j] -= (Summation[j] / (len(Y_train) % batchsize)) * LearningRateScaling
            else:
                for j in range(len(Summation)):
                    Coeff[j] -= (Summation[j] / batchsize) * LearningRateScaling
    return Coeff

In [16]:
alpha = [0.1,0.01,0.001,0.0001,0.00001,0.000001,0.0000001,0.00000001]
epoch = [5,50,500,5000,50000]

In [17]:
accuracy_batch =[]
for i in alpha:
    for j in epoch:
        batch_coeff=batchGradient(dataset_train,y_train,i,j)
        print('alpha',i)
        print('epoch',j)
        accuracy_batch.append(printaccuracy(dataset_test,y_test,batch_coeff))

Accuracy is : 83.33333333333334
Accuracy is : 83.33333333333334
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 90.0
Accuracy is : 83.33333333333334
Accuracy is : 83.33333333333334
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 93.33333333333333
Accuracy is : 90.0
Accuracy is : 83.33333333333334
Accuracy is : 83.33333333333334
Accuracy is : 86.66666666666667
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy is : 90.0
Accuracy is : 83.33333333333334
Accuracy is : 83.33333333333334
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy is : 90.0
Accuracy is : 83.33333333333334
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy is : 90.0
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy 

In [18]:
accuracy_stoch=[]
for i in alpha:
    for j in epoch:
        coeff_stoch = stochaisticGradient(dataset_train,y_train,i,j)
        print('alpha',i)
        print('epoch',j)
        accuracy_stoch.append(printaccuracy(dataset_test,y_test,coeff_stoch))

Accuracy is : 83.33333333333334
Accuracy is : 83.33333333333334
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 90.0
Accuracy is : 83.33333333333334
Accuracy is : 83.33333333333334
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 93.33333333333333
Accuracy is : 90.0
Accuracy is : 83.33333333333334
Accuracy is : 83.33333333333334
Accuracy is : 86.66666666666667
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy is : 90.0
Accuracy is : 83.33333333333334
Accuracy is : 83.33333333333334
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy is : 90.0
Accuracy is : 83.33333333333334
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy is : 90.0
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy is : 93.33333333333333
Accuracy 

In [19]:
accuracy_mini=[]
for i in alpha:
    for j in epoch:
        print('alpha',i)
        print('epoch',j)
        mini_coeff = minibtchgra(dataset_train,y_train,i,j)
        accuracy_mini.append(printaccuracy(dataset_test,y_test,mini_coeff))

Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy is : 86.66666666666667
Accuracy